In [ ]:
import datetime
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, segment_calcs,sched_rt_utils
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,  
    analysis_date,
)
from scripts import (A1_sjoin_vp_segments, A2_valid_vehicle_positions,B2_avg_speeds_by_segment)
import _threshold_utils as threshold_utils
import _rt_scheduled_utils as rt_scheduled_utils
CONFIG_PATH = './scripts/config.yml'
STOP_SEG_DICT = helpers.get_parameters(CONFIG_PATH, "stop_segments")

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
analysis_date

In [ ]:
dictionary = helpers.get_parameters(CONFIG_PATH, "stop_segments")

## % of Meters
* start with the speeds_stop_segments_{analysis_date} parquet (which is produced in B1_speeds_by_segment_trip ).
* grab in stop_segments_{analysis_date} (in CRS 3310 already), and you can get the segment's length.
* merge with the speeds by segment-trip, which contains the meters_elapsed column
* calculate pct where meters_elapsed/segment_length
* show me some charts around this, a couple of descriptives to make sure these are all ok
* give me rough descriptives of how many rows we're dropping if we go with keeping at least 30%, 40%, 50%

In [ ]:
def import_speeds_segs(analysis_date:str, 
                       max_speed_cutoff: int, 
                       dict_inputs:dict)-> pd.DataFrame:
    FILE = dict_inputs['stage4']
    df = pd.read_parquet(f"{SEGMENT_GCS}{FILE}_{analysis_date}", 
        filters = [[("speed_mph", "<=", max_speed_cutoff)]])
    return df

In [ ]:
def load_segments(analysis_date:str, dict_inputs:dict) -> gpd.GeoDataFrame:
    # Load in ALL segments, find the length
    FILE = dict_inputs['segments_file']
    segments = gpd.read_parquet(f"{SEGMENT_GCS}{FILE}_{analysis_date}.parquet")
    
    return segments

In [ ]:
segments_gdf = load_segments(analysis_date, dictionary)

In [ ]:
speeds = import_speeds_segs(analysis_date,70,  dictionary)

In [ ]:
def calculate_segment_length(segments:gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Find the length of segments
    """
    segments = segments.assign(
        meters_length=(segments.geometry.length)
    )
    #df = segments.drop(columns = ['geometry','district','district_name'])
    
    return segments

In [ ]:
def merge_segments_speeds(speeds: pd.DataFrame,
                          segments:gpd.GeoDataFrame, 
                          ) -> pd.DataFrame:
    """
    Merge speeds and segments, calculate length.
    """
    segments = calculate_segment_length(segments)
    
    merge_cols = ['shape_array_key','stop_sequence','schedule_gtfs_dataset_key']
    merge1 = pd.merge(segments, speeds, on = merge_cols, how = "inner")
    
    merge1['percent'] = merge1.meters_elapsed/merge1.meters_length * 100
    
    merge1.percent = merge1.percent.fillna(0)
    return merge1

In [ ]:
m1 = merge_segments_speeds(speeds, segments_gdf)

## Visualizing Speed
* https://nbviewer.org/github/cal-itp/data-analyses/blob/filter-speeds-avgs/rt_segment_speeds/18_speed_distribution.ipynb
* https://analysis.calitp.org/rt/district_07-los-angeles/9__speedmaps__district_07-los-angeles__itp_id_300.html

In [ ]:
def myround(x, base=5):
    return base * round(x/base)

In [ ]:
def valid_trips_by_cutoff(df, percentages:list):
    """
    Returns a table of how many trips/routes/operators/etc are kept 
    after applying a certain percentile cutoff for the 
    length of segment that's covered.
    
    Ex: if a segment has 40% of its length covered in RT 
    data, it falls in the 20th percentile.
    
    Args:
        df: result from  merge_segments_speeds(speeds, segments_gdf)
        percentages: the percentiles in floats like 0.2, 0.25, 0.3
    """
    final = pd.DataFrame()
    og_len = len(df)
    og_trips = df.trip_id.nunique()
    og_shape_array_key = df.shape_array_key.nunique()
    
    for i in percentages:
        # Round up percent to nearest 5. Ex: 43 becomes 45.
        percent = myround(df.percent.quantile(i).astype(float))
        temp = (df[(df.percent >= percent)])
        
        temp = temp.assign(
            percentile = f"Min. of {percent}% of seg length covered")
        
        temp = (temp
                .groupby(['percentile'])
                .agg({'gtfs_dataset_name':'count',
                      'speed_mph':'mean',
                      'shape_array_key':'nunique',
                      'trip_id':'nunique',
                      'gtfs_dataset_key':'nunique'})
                .reset_index()
                .rename(columns = {'gtfs_dataset_name':'n_rows',
                                  'speed_mph':'mean_speed_mph',
                                  'shape_array_key':'n_kept_routes',
                                  'trip_id':'n_kept_trips',
                                  'gtfs_dataset_key':'n_kept_operators'})
               )
        
        final = pd.concat([final, temp], axis=0)

    
    final = final.assign(
        percentage_kept_rows=final.n_rows.divide(og_len) * 100,
        percentage_kept_trips = final.n_kept_trips.divide(og_trips) * 100,
        percentage_kept_routes = final.n_kept_routes.divide(og_shape_array_key)*100)
    
    round_cols = ['percentage_kept_rows', 'percentage_kept_trips', 'percentage_kept_routes']
    final[round_cols] = final[round_cols].round(0)
    return final

### % of rows kept

In [ ]:
test = valid_trips_by_cutoff(m1, [.1,.2,.3,.4,.5,.6,.7])

In [ ]:
test = threshold_utils.pre_clean(test)

In [ ]:
test

In [ ]:
# Main chart
def bar_chart(df, x_column: str, y_column:str, title:str):
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(x=x_column, y= y_column, 
          color=alt.Color(y_column, 
          scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
          legend=None),
          tooltip = df.columns.tolist())
         .properties(title = title)
            )
    chart = threshold_utils.chart_size((chart), 400,300)
    return chart
    

In [ ]:
bar_chart(test, 'Percentage Kept Rows','Percentile', 'Rows Kept After % Segment Cutoff')

## Edited B2 function

In [ ]:
avg_test = B2_avg_speeds_by_segment.speeds_with_segment_geom(analysis_date, 
                                    70, 
                                    dictionary,
                                    0.40)

In [ ]:
STG5_FILE = dictionary['stage5']
og_avg = gpd.read_parquet(f"{SEGMENT_GCS}{STG5_FILE}_{analysis_date}.parquet")

In [ ]:
# See unique stops-keys-routes. They should have no more than 2.
avg_test.groupby(['shape_array_key','schedule_gtfs_dataset_key','stop_sequence']).agg({'district':'count'}).sort_values('district', ascending = False).head()

In [ ]:
og_avg.shape

In [ ]:
# 261593 w dropping dups at the end
avg_test.shape

In [ ]:
len(avg_test)-len(og_avg)

## Checks

### Seeing why some  rows were cut

In [ ]:
def look_at_missing_row(shape, sequence):
    display(og_avg.loc[(og_avg.shape_array_key == shape) & (og_avg.stop_sequence == sequence)].drop(columns = ['geometry']))
    display(m1.loc[(m1.shape_array_key == shape) & (m1.stop_sequence == sequence)][['percent', 'speed_mph']])
    display(avg_test.loc[(avg_test.shape_array_key == shape) & (avg_test.stop_sequence == sequence)].drop(columns = ['geometry']))

In [ ]:
missing_shape2  = "4d006755475d7c2a03c525995311ec16"
missing_stop_seq2 = 29

In [ ]:
look_at_missing_row(missing_shape2, missing_stop_seq2)

## Comparison between `avg_test` and the original file.

In [ ]:
# Results from original script
rt_segs_agg = (og_avg
            .groupby(['shape_array_key','schedule_gtfs_dataset_key'])
            .agg({'stop_sequence':'nunique'})
            .reset_index()
            .rename(columns = {'stop_sequence':'total_stops'})
            .add_prefix('og_')
           )

In [ ]:
# My test after filtering but adding back in cut rows
avg_test_agg = (avg_test
            .groupby(['shape_array_key','schedule_gtfs_dataset_key'])
            .agg({'stop_sequence':'nunique'})
            .reset_index()
            .rename(columns = {'stop_sequence':'total_stops'})
            .add_prefix('testing_')
           )

In [ ]:
avg_test_agg.shape, rt_segs_agg.shape

In [ ]:
m1 = (pd
      .merge(rt_segs_agg, avg_test_agg, 
             left_on = ['og_shape_array_key', 'og_schedule_gtfs_dataset_key'],
             right_on = ['testing_shape_array_key', 'testing_schedule_gtfs_dataset_key'], 
             how = 'outer', indicator = True)
     )

In [ ]:
m1['total_thrown_out_stops'] = m1.og_total_stops - m1.testing_total_stops

In [ ]:
m1.sample(3)

In [ ]:
m1._merge.value_counts()

In [ ]:
m1.total_thrown_out_stops.describe()

In [ ]:
m1.loc[m1.total_thrown_out_stops < -40]

In [ ]:
# avg_test.loc[avg_test.shape_array_key == "c8b41fafd9be5e579ad230bd28f74d17"].drop(columns = ['geometry'])

In [ ]:
# og_avg.loc[og_avg.shape_array_key == "c8b41fafd9be5e579ad230bd28f74d17"].drop(columns = ['geometry'])

In [ ]:
# merge1.loc[merge1.shape_array_key == "c8b41fafd9be5e579ad230bd28f74d17"][['stop_sequence','meters_elapsed','sec_elapsed','speed_mph','percent']].sort_values(by = ['stop_sequence'])

### Seeing all the sequences thrown out

In [ ]:
merge_cols = ['shape_array_key', 'stop_sequence', 'schedule_gtfs_dataset_key',
       'stop_id', 'loop_or_inlining']

In [ ]:
subset = ['shape_array_key', 'stop_sequence', 'schedule_gtfs_dataset_key',
       'stop_id', 'loop_or_inlining', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'time_of_day']

In [ ]:
og_avg2 = og_avg[subset]

In [ ]:
og_avg2.shape

In [ ]:
avg_test2 = avg_test[subset]

In [ ]:
avg_test2.shape

In [ ]:
m2 = pd.merge(og_avg2, avg_test2, on = merge_cols, how = 'outer', indicator = True)

In [ ]:
m2._merge.value_counts()

### Seeing which routes were thrown out

In [ ]:
og_routes = set(og_avg.shape_array_key.unique().tolist())
test_routes = set(avg_test.shape_array_key.unique().tolist())

new_routes = list(test_routes - og_routes)

In [ ]:
new_routes

In [ ]:
new_routes_df = avg_test.loc[avg_test.shape_array_key.isin(new_routes)]

In [ ]:
new_routes_df.shape_array_key.nunique()

In [ ]:
new_routes_df.shape

In [ ]:
new_routes_df.drop(columns = ['geometry'])

In [ ]:
type(new_routes_df)

In [ ]:
new_routes_df.explore('shape_array_key')